# CATE & Double Machine Learning

Prof. Daniel de Abreu Pereira Uhr

## Conteúdo

* CATEs - Conditional Average Treatment Effects (Efeitos Heterogêneos do Tratamento)
* Efeitos Heterogêneos do Tratamento e o Arcabouço de Resultados Potenciais
* Ortogonalização
  * Aplicação do Procedimento de Ortogonalização no Python
* Ortogonalização e Machine Learning  (Orthogonal/Double Machine Learning - DML)


## Referências

**Principais:**
* UBER CausalML: https://causalml.readthedocs.io/en/latest/
* Microsoft EconML: https://econml.azurewebsites.net/
* https://github.com/DoubleML/doubleml-for-py
* https://docs.doubleml.org/stable/index.html
* https://github.com/MasaAsami/ReproducingDMLDiD/blob/main/notebook/Reproduction_of_DMLDiD_RO_for_NEW_SIMDATA.ipynb
* https://econml.azurewebsites.net/
* Schmidheiny, K., & Siegloch, S. (2023). On event studies and distributed-lags in two-way fixed effects models: Identification, equivalence, and generalization. Journal of Applied Econometrics, 1- 19. https://doi.org/10.1002/jae.2971
* Stevenson, Betsey, Wolfers, Justin, 2006. Bargaining in the shadow of the law: Divorce laws and family distress. Q. J. Econ. 121 (1), 267–288.
* Goodman-Bacon, A. (2021). Difference-in-differences with variation in treatment timing. Journal of Econometrics. https://doi.org/10.1016/j.jeconom.2021.03.014
* Callaway, B. and Sant'Anna, P. H. C. (2021). Difference-in-Differences with multiple time periods. Journal of Econometrics. https://doi.org/10.1016/j.jeconom.2020.12.001

**Complementares:**

* Borusyak, K.; Jaravel, X. and Spiess, J. (2023). Revisiting Event Study Designs: Robust and Efficient Estimation. arXiv: https://arxiv.org/pdf/2108.12419.pdf
* Clément deChaisemartin, Xavier d’Haultfoeuille. (2022) Difference-in-Differences Estimators of Intertemporal Treatment Effects. hal-03873903
* Roth et al. (2022) What’s Trending in Difference-in-Differences? A Synthesis of the Recent Econometrics Literature. https://www.jonathandroth.com/assets/files/DiD_Review_Paper.pdf

## CATEs - Conditional Average Treatment Effects (Efeitos Heterogêneos do Tratamento)

A nossa função de expectativa condicional é dada por: 

$$ E[Y|X, T]$$

Suponha que queremos realizar inferência causal, entre $T$ e $Y$, sob um contexto $X$, e otimizá-la. Ou seja, 

$$ \text{argmax}_{T} E[Y|X, T]$$

Ou seja, agora queremos mais do que apenas o efeito médio do tratamento (ATE). Sabemos que o tratamento tem impacto positivo em algumas pessoas, mas não em outras. Os recursos $X$ (covariáveis) desempenham um papel na definição de diferentes perfis de unidades (indivíduos), e cada perfil (indivíduo) **pode responder de forma diferente ao tratamento**. Sendo assim, **agora queremos personalizar o tratamento, dando-o apenas às unidades que melhor respondem a ele** (a ideia é "direcionar o tratamento"). Por exemplo, se um medicamento tem efeitos colaterais graves para crianças, podemos querer restringir sua distribuição apenas para adultos. Ou se uma campanha publicitária é eficaz apenas em países de língua inglesa, não vale a pena mostrá-la em outro lugar.

**CATE**: 

O **Conditional Average Treatment Effect** (CATE) é a média do efeito do tratamento condicional a um conjunto de características. Para o caso de tratamento binário, temos: 

$$ E[Y_{1}-Y_{0}|X]$$

ou, para o caso de tratamento contínuo:

$$ E[y´(t)|X]$$

O condicionamento em $X$ significa que agora permitimos que o efeito do tratamento seja diferente dependendo das características de cada unidade (indivíduo). Queremos tratar apenas as unidades certas (no caso binário) ou descobrir qual é a dosagem de tratamento ideal para cada unidade (no caso contínuo).

Considere o exemplo gráfico:

<div style="text-align:center;">
    <img src="images\elast-partition.png"  alt="Imagem" style="width: 400px;"/>
</div>

Desagregando os dados, podemos ver que o efeito do tratamento é diferente para diferentes os 3 grupos de pessoas. 

<div style="text-align:center;">
    <img src="images\elast-split.png"  alt="Imagem" style="width: 800px;"/>
</div>

Assim, a ideia é buscar a indetificação do efeito heterogêneo do tratamento, para que possamos personalizar o tratamento para cada unidade. Vamos tentar entender esse mesmo gráfico de outra maneira. Suponha que queremos encontrar os dias em que a $ \frac{d \text{Vendas}}{d \text{Preço}} $ é menor. Isso significa que, para esses dias, a elasticidade-preço é menor. Logo, do ponto de vista do vendedor, poderíamos aumentar o preço nesses dias sem perder muitas vendas (será que isso ocorre no mercado de aluguéis de imóveis durante às férias?).

A elasticidade (sensibilidade do indivíduo) é não observável. Podemos pensar cada unidade como tendo um valor $Y_{i}$, com uma elasticidade (sensibilidade) individual ($ \frac{d Y_{i}}{d T} $). 

<div style="text-align:center;">
    <img src="images\elasticity.png"  alt="Imagem" style="width: 400px;"/>
</div>


Para vermos as inclinações individuais, teríamos que observar cada dia sob dois preços diferentes e calcular como as vendas mudam para cada um desses preços. 

$$ \frac{dY_{i}}{dT} = \frac{Y_{i}(T_{i}) - Y_{i}(T_{i} + \epsilon)}{T_{i}-(T_{i} + \epsilon)}$$

Este é o problema fundamental da inferência causal novamente. Nunca podemos ver a mesma unidade sob diferentes condições de tratamento. Então, o que podemos fazer?


Um possível ajuste aos dados é utilizar o OLS.

$$ y_{i} = \alpha + \beta T_{i} + \gamma X_{i} + \epsilon_{i} $$

Diferenciando o tratamento,

$$ \frac{dY_{i}}{dT} = \beta$$

Nesse caso, é um modelo simples, e um valor constante de beta para todos os indivíduos. Esse é o ATE. No entento, se fizermos a seguinte mudança simples:

$$ y_{i} = \alpha + \beta T_{i} + \gamma X_{i} + \delta X_{i}T_{i} + \epsilon_{i} $$

Teremos uma sensibilidade diferente para cada indivíduo:

$$ \frac{dY_{i}}{dT} = \beta + \delta X_{i}$$

Onde $\delta$ é um coeficiente que depende das características de cada indivíduo ($X_{i}$). Em outras palavras, a previsão de sensibilidade mudará conforme $X$. Com essas previsões de sensibilidade, podemos agrupar as unidades por quanto achamos que elas responderão ao tratamento.






## Efeitos Heterogêneos do Tratamento e o Arcabouço de Resultados Potenciais

Podemos definir o efeito do tratamento individual (Individual Treatment Effect - ITE - $\beta_{i}$) como a diferença entre os resultados potenciais. 

$$ 
\beta_{i}^{ITE} = Y_{i}(1) - Y_{i}(0) 
$$

ou, no caso do tratamento contínuo, $ \beta_{i}^{ITE} = dY_{i}/dt $, onde $t$ é a variável de tratamento. 

Segundo o problema fundamental da inferência causal, nunca podemos observar o mesmo indivíduo sob diferentes condições de tratamento. 

$$
Y^{obs}_i(t)= 
\begin{cases}
Y_i(1), & \text{se } t=1 \\
Y_i(0), & \text{se } t=0
\end{cases}
$$

Podemos definir o efeito médio do tratamento (Average Treatment Effect - ATE) como

$$
\beta^{ATE}= E[Y_i(1) − Y_i(0)] = E[\beta_i]
$$

e o efeito do tratamento médio condicional (Conditional Average Treatment Effect - CATE) como


$$ \beta^{CATE}(x) = E[Y_i(1) − Y_i(0)|X] = E[\beta_i|X_{i}=x] $$

**Os ITE são inerentemente não observáveis.**

O que pode ser estimado em vez disso é o **Conditional Average Treatment Effect (CATE)** , ou seja, o efeito esperado do tratamento individual, condicional em covariáveis $​​X$.

Para recuperar o CATE, precisamo fazer 3 suposições:

* Não Confundimento / Inconfundibilidade (Unconfoundedness): 

$$ Y_{i}(0), Y_{i}(1) \perp T|X $$

* Sobreposição (Overlap): 

$$ 0 < p(x) < 1 $$

* Consistência (Consistency): 

$$ Y_{i} = Y_{i}(1)T_{i} + Y_{i}(0)(1-T_{i}) $$

Onde $p(x)$ é o escore de propensão, ou seja, a probabilidade esperada de ser tratado, condicional às covariáveis ​​$X$.

Cabe destacar que os modelos lineares têm algumas desvantagens. A principal delas é a suposição de linearidade em $X$. Seria ótimo se pudéssemos substituir o modelo linear por um modelo de machine learning mais flexível. Poderíamos até mesmo conectar o tratamento como um recurso a um modelo de ML, como Decision Trees, rede neural ou Gradient Boosting. 

$$ y_{i} = M(X_{i}, T_{i}) + \epsilon_{i} $$


mas a partir daí, não está claro como podemos obter estimativas do efeito do tratamento, uma vez que este modelo produzirá previsões de $Y$, e não de $\beta$. Vamos aprender uns conceitos importantes para solucionar essa questão.





## Ortogonalização

Antes de estimar o CATE, precisamos conhecer um conceito importante: a Ortogonalização, e como ela é aplicada na econometria. A ideia de ortogonalização é baseada em um teorema elaborado por três econometristas em 1933, Ragnar Frisch, Frederick V. Waugh e Michael C. Lovell. Simplificando, afirma que você pode decompor qualquer modelo de regressão linear multivariável em três estágios ou modelos. 

Digamos que você tem uma matriz de covariáveis $X$, e voce particiona ela em duas partes, $X_{1}$ e $D$. 

* **Primeira Etapa**

Pegamos o primeiro conjunto de variáveis $X_{1}$ e fazemos uma regressão linear de $X_{1}$ em $Y$, onde $\theta_{1}$ é o vetor de parâmetros

$$ y_{i} = \theta_{0} + \theta_{1} X_{1i} + \epsilon_{i}$$

e guardamos os resíduos dessa regressão ($y^{*}$).

$$ y^{*}_{i} = y_{i} - \hat{y}_{i} = y_{i} - ( \hat{\theta}_{0} + \hat{\theta}_{1} X_{1i} )$$

* **Segunda Etapa**

Pegamos novamente o primeiro conjunto de características, mas agora executamos um modelo onde estimamos o segundo conjunto de características ($X_{2}$)

$$ D_{i} = \gamma(0) + \gamma(1) X_{1i} + e_{i}$$

Aqui, estamos usando o primeiro conjunto de recursos para prever o segundo conjunto de recursos. Por fim, consideramos também os resíduos desta segunda etapa.

$$ D_{i}^{*} = D_{i} - (\gamma(0) + \gamma(1) X_{1i})$$

* **Terceira etapa**

Por fim, pegamos os resíduos do primeiro e do segundo estágio e estimamos o seguinte modelo

$$ y_{i}^{*} = \beta_{0} + \beta_{2} D_{i}^{*} + e_{i}$$


* **Teorema Frisch – Waugh – Lovell (FWL)**

O teorema FWL afirma que a estimativa do parâmetro $\hat{\beta}_{2}$, estimado anteriormente, é equivalente ao que obtemos ao executar a regressão completa, com todas as covariáveis.

$$ y_{i} = \beta_{0} + \beta_{1} X_{1i} + \beta_{2} D_{i} + e_{i}$$


**Intuição do teorema FWL**

Sabemos que a regressão é um modelo muito especial. Cada um de seus parâmetros tem a interpretação de uma derivada parcial, quanto seria Y se X aumentasse em uma unidade, mantendo todos as outras covariáveis constantes. Sabemos também que se omitirmos variáveis ​​da regressão, teremos viés. Especificamente, viés variável omitido (ou viés de confusão). Ainda assim, Frisch-Waugh-Lovell está dizendo que posso dividir meu modelo de regressão em duas partes, nenhuma delas contendo o conjunto completo de recursos, e ainda assim obter a mesma estimativa que obteria executando a regressão inteira. 

O teorema fornece algumas dicas sobre o que a regressão linear está fazendo. Para obter o coeficiente de uma variável $X_{k}$, a regressão primeiro usa todas as outras variáveis ​​para prever $X_{k}$ e pega os resíduos. Isso “limpa” $X_{k}$ de qualquer influência dessas variáveis. Dessa forma, quando tentamos entender o impacto de $X_{k}$ sobre $Y$, estará livre de viés de variável omitida. Em segundo lugar, a regressão usa todas as outras variáveis ​​para prever $Y$ e pega os resíduos. Isso “limpa” $Y$ de qualquer influência dessas variáveis, reduzindo a variância de $Y$ para que seja mais fácil ver como $X_{k}$ afeta $Y$.

A regressão linear está estimando o impacto de $D$ sobre $y$ enquanto contabiliza $X_{1}$. Isso é importante para inferência causal. 

Assim, podemos construir um modelo que preveja um tratamento ($T$) usando as covariáveis $X$, um modelo que prevê o resultado $y$ usando as mesmas covariáveis, pega os resíduos de ambos os modelos e executa um modelo que estima como o resíduo de $T$ afeta o resíduo de $y$. Este último modelo vai me dizer como $T$ afeta $y$ enquanto controla por $X$. Ou seja, os dois primeiros modelos controlam as variáveis de confusão. Eles estão gerando dados que são praticamente aleatórios. Isso está distorcendo meus dados. É isso que usamos no modelo final para estimar a elasticidade.

### Aplicação do Procedimento de Ortogonalização no Python

Vamos aplicar o procedimento de ortogonalização considerando um modelo de regressão linear simples. Vamos realizar a orgonalização supondo linearidade entre as variáveis para entender o conceito. Posteriormente, vamos aplicar o procedimento de ortogonalização em um modelo de machine learning. 

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import statsmodels.formula.api as smf
import statsmodels.api as sm

In [3]:
# DataFrame
df = pd.read_stata("https://github.com/Daniel-Uhr/data/raw/main/cattaneo2.dta")

# Criar a variável de resultado
df['Y'] = df['bweight']

# Criar a variável 'Treated' com valor 1 se 'mbsmoke' for 'smoker', caso contrário 0
df['Treated'] = np.where(df['mbsmoke'] == 'smoker', 1, 0)

# Criar a variável 'casada' com valor 1 se 'mmarried' for 'married', caso contrário 0
df['casada'] = np.where(df['mmarried'] == 'married', 1, 0)


Para desviar este conjunto de dados, precisaremos de dois modelos. O primeiro modelo, vamos chamá-lo $M_{t}(X)$, prevê o tratamento (Se a gestante é fumante, no nosso caso) utilizando os confundidores. É um dos estágios que vimos acima, no teorema de Frisch–Waugh–Lovell.

In [4]:
m_D = smf.ols("Treated ~ 1 + casada + mage + medu + fhisp + mhisp + foreign + alcohol + deadkids + nprenatal + mrace + frace + fage + fedu", data=df).fit()
df['m_D_star'] = df['Treated'] - m_D.predict(df)

Assim que tivermos este modelo, construiremos os resíduos

$$ \hat{D}_{i} = D_{i} - M_{t}(X_{i})$$

Você pode pensar neste resíduo como uma versão do tratamento que é imparcial ou, melhor ainda, que é impossível de prever a partir dos fatores de confusão $X$. Como os fatores de confusão já eram usados ​​para prever $t$, o resíduo é, por definição, imprevisível com com $X$. Outra maneira de dizer isso é que o viés foi explicado pelo modelo $M_{t}(X)$, produzindo $\hat{t}_{i}$ que é tão bom quanto atribuído aleatoriamente. É claro que isso só funciona se tivermos em $X$ todos os fatores de confusão que causam ambos $T$ e $Y$.

Também podemos construir resíduos para o resultado.

$$ \hat{y}_{i} = y_{i} - M_{y}(X_{i})$$


Este é outro estágio do teorema de Frisch – Waugh – Lovell. Isso não torna o conjunto menos tendencioso, mas facilita a estimativa do efeito, reduzindo a variância em $y$. Mais uma vez você pode pensar $\hat{y}_{i}$ como uma versão de $y_{i}$ imprevisível de $X$ ou que teve todas as suas variações devido a $X$ explicadas. Pense nisso. Nós já usamos $X$ para prever $y$ com $M_{y}(X_{i})$. E $\hat{y}_{i}$ é o erro dessa previsão. Então, por definição, não é possível prever isso a partir de $X$. Todas as informações em $X$ para prever $y$ já foram usadas. Se for esse o caso, a única coisa que resta para explicar $\hat{y}_{i}$ é algo que não usamos usamos para construí-lo (não incluído em $X$), que é apenas o tratamento (novamente, assumindo que não há fatores de confusão não medidos).


In [5]:
m_y = smf.ols("Y ~  1 + casada + mage + medu + fhisp + mhisp + foreign + alcohol + deadkids + nprenatal + mrace + frace + fage + fedu", data=df).fit()
df['m_y_star'] = df['Y'] - m_y.predict(df)

Considerando o OLS tradicional com covariáveis.

In [6]:
ols = smf.ols("Y ~ Treated + 1 + casada + mage + medu + fhisp + mhisp + foreign + alcohol + deadkids + nprenatal + mrace + frace + fage + fedu", data=df).fit()
print(ols.summary())

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.104
Model:                            OLS   Adj. R-squared:                  0.102
Method:                 Least Squares   F-statistic:                     38.49
Date:                Mon, 28 Oct 2024   Prob (F-statistic):          5.80e-100
Time:                        14:00:17   Log-Likelihood:                -35858.
No. Observations:                4642   AIC:                         7.175e+04
Df Residuals:                    4627   BIC:                         7.184e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   2851.5203     54.983     51.861      0.0

Agora vamos verificar se teorema de Frisch-Waugh-Lovell funciona mesmo.

In [7]:
FWL1 = smf.ols("m_y_star ~ m_D_star", data=df).fit()
print(FWL1.summary())

                            OLS Regression Results                            
Dep. Variable:               m_y_star   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                  0.021
Method:                 Least Squares   F-statistic:                     100.6
Date:                Mon, 28 Oct 2024   Prob (F-statistic):           1.91e-23
Time:                        14:00:19   Log-Likelihood:                -35858.
No. Observations:                4642   AIC:                         7.172e+04
Df Residuals:                    4640   BIC:                         7.173e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   2.348e-11      8.041   2.92e-12      1.0

Depois de fazermos as duas transformações, a única coisa que resta para prever esses resíduos é o tratamento. 

Para resumir, ao prever o tratamento, construímos $\hat{t}$ que funciona como uma versão imparcial do tratamento; ao prever o resultado, construímos $\hat{y}$ que é uma versão do resultado que só pode ser explicada se usarmos o tratamento. Esses dados, onde substituímos por $y$ por $\hat{y}$ e $t$ por $\hat{t}$, são os dados desviados que queríamos. Podemos usá-lo para avaliar nosso modelo causal da mesma forma que fizemos anteriormente, usando dados aleatórios.

###  DML - Orthogonal/Double Machine Learning

* Double Machine Learning - DML
* Double/Debiased Machine Learning - DDML 

Quando temos muitas possíveis variáveis ​​de controle, podemos querer selecionar as mais relevantes, possivelmente capturando não linearidades e interações. Algoritmos de aprendizado de máquina são perfeitos para essa tarefa. No entanto, nesses casos, estamos introduzindo um viés que é chamado de regularização ou pré-teste, ou viés de seleção de recursos ($X$). 

Ou seja, o que acontece se a dimensão de $X$ aumenta e não conhecemos a forma funcional através da qual $X$ afeta $Y$ e $D$?

Nesses casos, podemos usar **algoritmos de aprendizado de máquina** para **descobrir essas relações não lineares de alta dimensão**.

No artigo de **Chernozhukov et al (2018)**, os autores mostraram que também é possível fazer ortogonalização com modelos de aprendizado de máquina. 

$$ \hat{y}_{i} = y_{i} - M_{y}(X_{i})$$

$$ \hat{D}_{i} = D_{i} - M_{t}(X_{i})$$


***Machine Learning (ML)* e *Overfitting***

Os modelos de aprendizado de máquina (ML) podem ajustar-se perfeitamente aos dados, ou melhor, superajustá-los (*Overfitting* / "sobreajuste"). 

  * Apenas olhando para as equações anteriores, podemos saber o que acontecerá nesse caso. Se $M_{y}$ de alguma forma, os resíduos serão todos muito próximos de zero. Se isso acontecer, será difícil descobrir como $t$ afeta isso. 

  * Da mesma forma, se $M_{t}$ de alguma forma superajusta, seus resíduos também serão próximos de zero. Conseqüentemente, não haverá variação no resíduo do tratamento para ver como isso pode impactar o resultado.

  * OBS1: O que podemos fazer para evitar isso o overfitting? A solução é simples: **regularização**. A regularização é uma técnica que adiciona um termo à função de perda que penaliza os coeficientes do modelo. Isso faz com que o modelo seja menos sensível aos dados de treinamento, evitando o superajuste.
  * OBS2: Os modelos mais comuns de ML para previsão são: Random Forest, Gradient Boosting, Redes Neurais, Support Vector Machines, etc.
    * Random Forest: é um modelo de aprendizado de máquina que pode ser usado tanto para classificação quanto para regressão. Ele é um modelo de conjunto que treina várias árvores de decisão em subconjuntos aleatórios dos dados e faz a média de suas previsões.
    * Gradient Boosting: é um modelo de aprendizado de máquina que constrói um modelo aditivo de forma progressiva. Ele permite a otimização de funções de perda diferenciáveis arbitrárias.
    * Redes Neurais: são modelos de aprendizado de máquina que são inspirados na forma como o cérebro humano funciona. Eles são compostos por camadas de neurônios que processam e transmitem informações.
    * Support Vector Machines: são modelos de aprendizado de máquina que são usados para classificação e regressão. Eles são eficazes em espaços de alta dimensão e são capazes de lidar com dados não lineares.
    * Outros modelos de ML mais usados em economia são: LASSO, Ridge, Elastic Net, etc. LASSO é um método de regressão que adiciona uma penalidade L1 à função de perda. Isso faz com que alguns coeficientes sejam exatamente zero, o que é útil para seleção de recursos. Ridge é um método de regressão que adiciona uma penalidade L2 à função de perda. Isso faz com que os coeficientes sejam menores, o que é útil para reduzir a variância. Elastic Net é um método de regressão que combina as penalidades L1 e L2. Isso permite que você selecione recursos e reduza a variância ao mesmo tempo.

**Validação Cruzada (Cross-Validation)**

Para explicar esse procedimemto, precisamos fazer a divisão da amostra. Ou seja, estimamos o modelo com uma parte do conjunto de dados e fazemos previsões na outra parte. Uma maneira simples e intuitiva seria dividir a amostra de teste ao meio, fazer dois modelos de forma que cada um seja estimado em uma metade do conjunto de dados e faça previsões na outra metade.

Já uma implementação um pouco mais elegante usa a chamada **validação cruzada K-fold**. Ist



A vantagem é que podemos treinar todos os modelos em uma amostra maior que metade do conjunto de teste.

<div style="text-align:center;">
    <img src="images\kfold-cv.png"  alt="Imagem" style="width: 450px;"/>
</div>

Felizmente, esse tipo de previsão cruzada é muito fácil de implementar usando `cross_val_predicta` função do Sklearn.


Vamos utilizar o Random Forest para fazer a previsão do tratamento e do resultado.

In [8]:
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import RandomForestRegressor

X = ['casada', 'mage', 'medu', 'fhisp', 'mhisp', 'foreign', 'alcohol', 'deadkids', 'nprenatal', 'mrace', 'frace', 'fage', 'fedu']
D = "Treated"
y = "Y"

folds = 5

np.random.seed(123)
m_D = RandomForestRegressor(n_estimators=100)
D_res1 = df[D] - cross_val_predict(m_D, df[X], df[D], cv=folds)

m_y = RandomForestRegressor(n_estimators=100)
y_res1 = df[y] - cross_val_predict(m_y, df[X], df[y], cv=folds)

Agora que temos os resíduos, vamos armazená-los como colunas em um novo conjunto de dados.

In [9]:
DML1 = df.assign(**{
    "Y-ML_y(X)": y_res1,
    "Treated-ML_t(X)": D_res1,
})
DML1.head()

,bweight,mmarried,mhisp,fhisp,foreign,alcohol,deadkids,mage,medu,fage,...,lbweight,fbaby,prenatal1,Y,Treated,casada,m_D_star,m_y_star,Y-ML_y(X),Treated-ML_t(X)
0,3459,married,0,0,0,0,0,24,14,28,...,0,No,Yes,3459,0,1,-0.086026,47.203835,-295.130000,-0.026667
1,3260,notmarried,0,0,1,0,0,20,10,0,...,0,No,Yes,3260,0,0,-0.315814,277.930277,303.140000,-0.410000
2,3572,married,0,0,1,0,0,22,9,30,...,0,No,Yes,3572,0,1,-0.186338,196.895284,132.310000,-0.070000
3,2948,married,0,0,0,0,0,26,12,30,...,0,No,Yes,2948,0,1,-0.156887,-451.977522,-460.669000,0.000000
4,2410,married,0,0,0,0,0,20,12,21,...,1,Yes,Yes,2410,0,1,-0.121311,-1040.633348,-1350.973333,0.000000


In [10]:
FWL_DML1 = smf.ols("y_res1 ~ D_res1", data=DML1).fit()
print(FWL_DML1.summary())

                            OLS Regression Results                            
Dep. Variable:                 y_res1   R-squared:                       0.014
Model:                            OLS   Adj. R-squared:                  0.014
Method:                 Least Squares   F-statistic:                     66.82
Date:                Mon, 28 Oct 2024   Prob (F-statistic):           3.80e-16
Time:                        14:00:42   Log-Likelihood:                -36189.
No. Observations:                4642   AIC:                         7.238e+04
Df Residuals:                    4640   BIC:                         7.239e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      4.1388      8.646      0.479      0.6

Vejamos outro exemplo do DML. Considerando Gradient Boosting Machines (GBM) para prever o tratamento e o resultado.

In [11]:
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import GradientBoostingRegressor
import numpy as np

# Definir as variáveis X, D, y
X = ['casada', 'mage', 'medu', 'fhisp', 'mhisp', 'foreign', 'alcohol', 'deadkids', 'nprenatal', 'mrace', 'frace', 'fage', 'fedu']
D = "Treated"
y = "Y"

# Definir o número de folds para a validação cruzada
folds = 5

# Garantir reprodutibilidade nos modelos de Gradient Boosting
m_D = GradientBoostingRegressor(n_estimators=100, random_state=123)
D_res2 = df[D] - cross_val_predict(m_D, df[X], df[D], cv=folds)

m_y = GradientBoostingRegressor(n_estimators=100, random_state=123)
y_res2 = df[y] - cross_val_predict(m_y, df[X], df[y], cv=folds)


In [12]:
DML2 = df.assign(**{
    "Y-ML_y(X)": y_res2,
    "Treated-ML_t(X)": D_res2,
})
DML2.head()

,bweight,mmarried,mhisp,fhisp,foreign,alcohol,deadkids,mage,medu,fage,...,lbweight,fbaby,prenatal1,Y,Treated,casada,m_D_star,m_y_star,Y-ML_y(X),Treated-ML_t(X)
0,3459,married,0,0,0,0,0,24,14,28,...,0,No,Yes,3459,0,1,-0.086026,47.203835,44.846311,-0.065141
1,3260,notmarried,0,0,1,0,0,20,10,0,...,0,No,Yes,3260,0,0,-0.315814,277.930277,173.141492,-0.379986
2,3572,married,0,0,1,0,0,22,9,30,...,0,No,Yes,3572,0,1,-0.186338,196.895284,84.286938,-0.182789
3,2948,married,0,0,0,0,0,26,12,30,...,0,No,Yes,2948,0,1,-0.156887,-451.977522,-412.680405,-0.171932
4,2410,married,0,0,0,0,0,20,12,21,...,1,Yes,Yes,2410,0,1,-0.121311,-1040.633348,-1096.367541,-0.070470


In [13]:
FWL_DML2 = smf.ols("y_res2 ~ D_res2", data=DML2).fit()
print(FWL_DML2.summary())

                            OLS Regression Results                            
Dep. Variable:                 y_res2   R-squared:                       0.019
Model:                            OLS   Adj. R-squared:                  0.018
Method:                 Least Squares   F-statistic:                     88.46
Date:                Mon, 28 Oct 2024   Prob (F-statistic):           7.93e-21
Time:                        14:00:50   Log-Likelihood:                -35882.
No. Observations:                4642   AIC:                         7.177e+04
Df Residuals:                    4640   BIC:                         7.178e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.7775      8.082      0.096      0.9

Vimos a mecânica do DML. 

Mas qual o melhor modelo de ML para prever o tratamento e o resultado? 

Precisamos avaliar, um critério de decisão comum é utilizar o erro quadrático médio (MSE) dos resíduos gerados para $D$ e $Y$ para ambos os modelos, e então comparar seus desempenhos.

In [14]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# Definir as variáveis
X = ['casada', 'mage', 'medu', 'fhisp', 'mhisp', 'foreign', 'alcohol', 'deadkids', 'nprenatal', 'mrace', 'frace', 'fage', 'fedu']
D = "Treated"
y = "Y"
folds = 5

# RandomForest - Estimando m_D e m_y
rf_model_D = RandomForestRegressor(n_estimators=100)
D_res_rf = df[D] - cross_val_predict(rf_model_D, df[X], df[D], cv=folds)

rf_model_y = RandomForestRegressor(n_estimators=100)
y_res_rf = df[y] - cross_val_predict(rf_model_y, df[X], df[y], cv=folds)

# GradientBoosting - Estimando m_D e m_y
gb_model_D = GradientBoostingRegressor(n_estimators=100)
D_res_gb = df[D] - cross_val_predict(gb_model_D, df[X], df[D], cv=folds)

gb_model_y = GradientBoostingRegressor(n_estimators=100)
y_res_gb = df[y] - cross_val_predict(gb_model_y, df[X], df[y], cv=folds)

# Calculando o MSE para cada modelo
mse_D_rf = mean_squared_error(df[D], df[D] - D_res_rf)  # Random Forest para D
mse_y_rf = mean_squared_error(df[y], df[y] - y_res_rf)  # Random Forest para y

mse_D_gb = mean_squared_error(df[D], df[D] - D_res_gb)  # Gradient Boosting para D
mse_y_gb = mean_squared_error(df[y], df[y] - y_res_gb)  # Gradient Boosting para y

# Exibir os resultados de MSE
print("MSE for D - Random Forest:", mse_D_rf)
print("MSE for y - Random Forest:", mse_y_rf)
print("MSE for D - Gradient Boosting:", mse_D_gb)
print("MSE for y - Gradient Boosting:", mse_y_gb)


MSE for D - Random Forest: 0.15049072305334754
MSE for y - Random Forest: 350812.5710231999
MSE for D - Gradient Boosting: 0.1322742607784923
MSE for y - Gradient Boosting: 309191.01191423537


Comparação para $D$ (Variável de Tratamento):

* Random Forest MSE: 0.1506
* Gradient Boosting MSE: 0.1323

Aqui, o Gradient Boosting tem um desempenho melhor na predição de $D$, pois seu MSE é menor. Isso significa que ele foi mais eficiente na captura da relação entre as variáveis explicativas $X$ e o tratamento $D$, resultando em resíduos menores.

Comparação para $Y$
* Random Forest MSE: 352060.35
* Gradient Boosting MSE: 309063.98

Novamente, o Gradient Boosting apresenta um MSE menor na predição de $y$, indicando que ele conseguiu capturar melhor a relação entre as variáveis explicativas  $X$ e o resultado $Y$, quando comparado ao Random Forest.


Agora vamos fazer de forma geral a análise para diversos modelos e verificar qual o melhor baseados no critério de erro quadrado médio. 

In [15]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR

# Definir os modelos que vamos testar
models = {
    "RandomForest": RandomForestRegressor(n_estimators=100),
    "GradientBoosting": GradientBoostingRegressor(n_estimators=100),
    "Lasso": Lasso(alpha=0.1),
    "Ridge": Ridge(alpha=1.0),
    "ElasticNet": ElasticNet(alpha=0.1, l1_ratio=0.5),
    "MLP": MLPRegressor(hidden_layer_sizes=(50,), max_iter=1000),
    "SVR": SVR()
}

# Definir as variáveis
X = ['casada', 'mage', 'medu', 'fhisp', 'mhisp', 'foreign', 'alcohol', 'deadkids', 'nprenatal', 'mrace', 'frace', 'fage', 'fedu']
D = "Treated"
y = "Y"
folds = 5

# Dicionário para armazenar os MSE de cada modelo para D e y
mse_results = {}

# Loop para ajustar os modelos e calcular os MSE para D e y
for name, model in models.items():
    # Estimativa de D com validação cruzada
    D_res = df[D] - cross_val_predict(model, df[X], df[D], cv=folds)
    # Estimativa de y com validação cruzada
    y_res = df[y] - cross_val_predict(model, df[X], df[y], cv=folds)
    
    # Calcular MSE para D e y
    mse_D = mean_squared_error(df[D], df[D] - D_res)
    mse_y = mean_squared_error(df[y], df[y] - y_res)
    
    # Armazenar os resultados
    mse_results[name] = {"MSE for D": mse_D, "MSE for y": mse_y}

# Exibir os resultados
for model_name, mse in mse_results.items():
    print(f"Model: {model_name}")
    print(f"  MSE for D: {mse['MSE for D']}")
    print(f"  MSE for y: {mse['MSE for y']}\n")


Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.


Model: RandomForest
  MSE for D: 0.15139084627531682
  MSE for y: 349909.7289144392

Model: GradientBoosting
  MSE for D: 0.1322770984849905
  MSE for y: 309273.4665657201

Model: Lasso
  MSE for D: 0.14514535464451975
  MSE for y: 308621.37257196667

Model: Ridge
  MSE for D: 0.13530450869509936
  MSE for y: 308669.9810617438

Model: ElasticNet
  MSE for D: 0.14369145438382802
  MSE for y: 308710.90380429785

Model: MLP
  MSE for D: 0.1431742709965872
  MSE for y: 363171.59636381926

Model: SVR
  MSE for D: 0.1582687258940923
  MSE for y: 330887.97070007736



Para os resíduos de $D$: O Gradient Boosting é o modelo preferido, pois ele obteve o menor MSE para prever a variável de tratamento.

Para os resíduos de $y$: O Lasso foi o melhor em prever o resultado.

Vou aumentar o número fold para 8 e verificar se os resultados mudam. Esse aumento de 5 para 8 folds é importante para garantir que o modelo seja treinado em uma amostra maior, o que pode melhorar a precisão das previsões. Além de ser uma prática para verificar a robustez dos resultados.

In [20]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR

# Definir os modelos que vamos testar
models = {
    "RandomForest": RandomForestRegressor(n_estimators=100),
    "GradientBoosting": GradientBoostingRegressor(n_estimators=100),
    "Lasso": Lasso(alpha=0.1),
    "Ridge": Ridge(alpha=1.0),
    "ElasticNet": ElasticNet(alpha=0.1, l1_ratio=0.5),
    "MLP": MLPRegressor(hidden_layer_sizes=(50,), max_iter=1000),
    "SVR": SVR()
}

# Definir as variáveis
X = ['casada', 'mage', 'medu', 'fhisp', 'mhisp', 'foreign', 'alcohol', 'deadkids', 'nprenatal', 'mrace', 'frace', 'fage', 'fedu']
D = "Treated"
y = "Y"
folds = 8

# Dicionário para armazenar os MSE de cada modelo para D e y
mse_results = {}

# Loop para ajustar os modelos e calcular os MSE para D e y
for name, model in models.items():
    # Estimativa de D com validação cruzada
    D_res = df[D] - cross_val_predict(model, df[X], df[D], cv=folds)
    # Estimativa de y com validação cruzada
    y_res = df[y] - cross_val_predict(model, df[X], df[y], cv=folds)
    
    # Calcular MSE para D e y
    mse_D = mean_squared_error(df[D], df[D] - D_res)
    mse_y = mean_squared_error(df[y], df[y] - y_res)
    
    # Armazenar os resultados
    mse_results[name] = {"MSE for D": mse_D, "MSE for y": mse_y}

# Exibir os resultados
for model_name, mse in mse_results.items():
    print(f"Model: {model_name}")
    print(f"  MSE for D: {mse['MSE for D']}")
    print(f"  MSE for y: {mse['MSE for y']}\n")

c:\Users\danie\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\danie\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\danie\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\danie\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the 

Model: RandomForest
  MSE for D: 0.1520980970340551
  MSE for y: 349468.8053384342

Model: GradientBoosting
  MSE for D: 0.13244114694873113
  MSE for y: 308077.9936950858

Model: Lasso
  MSE for D: 0.14517956178493543
  MSE for y: 308414.018017211

Model: Ridge
  MSE for D: 0.13535137678662412
  MSE for y: 308426.02969660814

Model: ElasticNet
  MSE for D: 0.1437445145464821
  MSE for y: 308393.73652545904

Model: MLP
  MSE for D: 0.14012366416355287
  MSE for y: 352474.1787789329

Model: SVR
  MSE for D: 0.1582865914491929
  MSE for y: 330487.8627229285



Aumentando para 8 folds, o Gradient Boosting torna-se o melhor modelo para prever tanto o tratamento quanto o resultado.

In [14]:
import pandas as pd
from econml.dml import LinearDML
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingRegressor


In [16]:
# Definir as variáveis
X = df[['casada', 'mage', 'medu', 'fhisp', 'mhisp', 'foreign', 'alcohol', 'deadkids', 'nprenatal', 'mrace', 'frace', 'fage', 'fedu']]
D = df['Treated']
y = df['Y']

# Converter variáveis categóricas em dummies (se necessário)
X = pd.get_dummies(X, drop_first=True)

# Dividir os dados em treino e teste (opcional, pois o EconML faz validação cruzada internamente)
# X_train, X_test, D_train, D_test, y_train, y_test = train_test_split(X, D, y, test_size=0.2, random_state=123)

# Definir os modelos de Machine Learning para o modelo condicional de tratamento e resultado
model_y = RandomForestRegressor(n_estimators=100, random_state=123)
model_t = RandomForestClassifier(n_estimators=100, random_state=123)

# Criar o estimador LinearDML
estimator = LinearDML(model_y=model_y,
                      model_t=model_t,
                      discrete_treatment=True,
                      random_state=123)

# Ajustar o modelo
estimator.fit(y, D, X=X)

# Obter o efeito causal médio
ate = estimator.ate(X=X)
print("Estimativa do Efeito Causal Médio (ATE):", ate)

# Obter o intervalo de confiança
ate_interval = estimator.ate_interval(X=X)
print("Intervalo de Confiança para o ATE:", ate_interval)

# Obter o efeito heterogêneo (se necessário)
# te = estimator.effect(X=X)
# print("Efeito Tratamento Heterogêneo:", te)


Estimativa do Efeito Causal Médio (ATE): -185.54586945953977
Intervalo de Confiança para o ATE: (-237.53567774646416, -133.5560611726154)


## Estimação do CATE com DML

Até agora, vimos como o Double/Debiased ML (Double Machine Learning - DML) nos permite focar na estimativa do Efeito Médio do Tratamento (ATE). No entanto, ele também pode ser usado para estimar a heterogeneidade dos efeitos do tratamento ou o Efeito Médio Condicional do Tratamento (CATE). 

Logo, o CATE para modelos DoubleMLPLR consideram uma versão ligeiramente ajustada do modelo DoubleMLPLR. Em vez de considerar um efeito de tratamento constante para todas as observações, o modelo ajustado permite um efeito diferente com base em grupos.

$$ Y = D \theta_{0} (X) + g_{0}(X) + \epsilon$$

com  $E(\epsilon|D,X)=0$, e

$$ D = m_{0}(X) + \eta$$

com $E(\eta|X)=0$.

onde $\theta_{0}(X)$ é o efeito heterogêneo do tratamento.

Ou seja, o CATE $\Theta_{ij}(X)$ tem a seguinte forma:

$$ \Theta_{i}(X) = X' coef_{ij} + cate\_intercept_{ij}$$

Coefficient Results table portrays the $coef_{ij}$ parameter vector for each outcome $i$ and treatment $j$. Intercept Results table portrays the $cate\_intercept_{ij}$ parameter.

In [6]:
print(estimator.summary())

                      Coefficient Results                       
          point_estimate  stderr zstat  pvalue ci_lower ci_upper
----------------------------------------------------------------
casada             8.846  57.364  0.154  0.877 -103.584  121.277
mage             -11.047   5.421 -2.038  0.042  -21.672   -0.423
medu              26.091  10.723  2.433  0.015    5.074   47.107
fhisp              21.56 126.313  0.171  0.864 -226.009  269.129
mhisp            -164.88 144.534 -1.141  0.254 -448.161  118.402
foreign           435.58  132.44  3.289  0.001  176.003  695.158
alcohol          -20.385  95.108 -0.214   0.83 -206.793  166.022
deadkids          41.641  55.025  0.757  0.449  -66.206  149.489
nprenatal         -8.174   7.046  -1.16  0.246  -21.983    5.635
mrace           -143.311 103.668 -1.382  0.167 -346.496   59.874
frace            -19.611  97.655 -0.201  0.841  -211.01  171.789
fage              -0.933   2.945 -0.317  0.751   -6.705    4.838
fedu              -1.335 

**Interpretação:**

* mage (idade da mãe):
  * point_estimate: -11.047 (pvalue: 0.042)
  * Isso sugere que, para cada aumento de um ano na idade da mãe, o efeito negativo de fumar durante a gravidez no peso ao nascer aumenta em 11.047 gramas (ou seja, o efeito se torna mais negativo).
  * Há evidência estatística de que a idade da mãe influencia o efeito de fumar durante a gravidez sobre o peso ao nascer.

* medu (educação materna):
  * point_estimate: 26.091 (pvalue: 0.015)
  * Para cada ano adicional de educação da mãe, o efeito negativo de fumar durante a gravidez no peso ao nascer é reduzido em 26.091 gramas (o efeito negativo é mitigado).
  * A educação materna parece reduzir o impacto negativo de fumar durante a gravidez.

* foreign (se a mãe é estrangeira):
  * point_estimate: 435.58 (pvalue: 0.001 (altamente significativo))
  * Interpretation: Mães estrangeiras têm um efeito tratamento condicional que é 435.58 gramas maior do que o de mães não estrangeiras.
  * A origem estrangeira da mãe está associada a uma redução significativa do efeito negativo de fumar durante a gravidez.

* Outras variáveis: Algumas covariáveis não são estatisticamente significativas (pvalue > 0.05), indicando que não há evidência suficiente para afirmar que essas covariáveis influenciam o efeito do tratamento.

* CATE Intercept Results (Resultados do Intercepto do CATE):
  * cate_intercept: 7.624
  * Este é o valor base do efeito tratamento condicional quando todas as covariáveis estão em zero. Como zero pode não ser um valor interpretável para algumas covariáveis (por exemplo, idade da mãe), o intercepto isoladamente pode não ter uma interpretação prática direta.

* OBS: Covariáveis contínuas vs. categóricas: Para variáveis contínuas (como mage), o coeficiente representa a variação no efeito do tratamento por unidade de aumento na covariável. Para variáveis binárias (como foreign), o coeficiente representa a diferença no efeito do tratamento entre os grupos (por exemplo, estrangeiras vs. não estrangeiras).


**Resumo:**

* O modelo estima que o efeito do tratamento (fumar durante a gravidez) sobre o peso ao nascer não é constante, mas varia linearmente com as covariáveis $X$. O sinal e magnitude dos coeficientes indicam a direção e a intensidade com que cada covariável afeta o efeito do tratamento.
* O impacto de fumar durante a gravidez no peso ao nascer não é o mesmo para todas as mães; varia de acordo com características como idade, educação e nacionalidade.
* Os resultados sugerem que políticas públicas visando reduzir o tabagismo durante a gravidez podem ser mais eficazes se levarem em consideração essas características. Por exemplo, focar em mães mais jovens ou com menor nível educacional.

A linear parametric conditional average treatment effect (CATE) model was fitted:

$$ Y = \Theta(X)\cdot T + g(X, W) + \epsilon $$

where for every outcome $i$ and treatment $j$ the CATE $\Theta_{ij}(X)$ has the form:
$$ \Theta_{ij}(X) = X' coef_{ij} + cate\_intercept_{ij}$$

Coefficient Results table portrays the $coef_{ij}$ parameter vector for each outcome $i$ and treatment $j$. Intercept Results table portrays the $cate\_intercept_{ij}$ parameter.</sub>

In [2]:
# Obter o efeito heterogêneo (se necessário)
te = estimator.effect(X=X)
print("Efeito Tratamento Heterogêneo:", te)


Efeito Tratamento Heterogêneo: [-175.54672274  434.12075885  159.15035232 ...  -47.47973106 -224.89001684
 -214.77611673]


Podemos utilizar diretamente ou o pacote EconML, ou DoubleML, que é uma biblioteca Python para estimar e inferir sobre os efeitos heterogêneos do tratamento. 

Cabe destacar que o DoubleML estima:
* Partially linear regression (PLR) - DoubleMLPLR
* Partially linear IV regression models (PLIV) - DoubleMLPLIV
* Interactive regression models (IRM) - DoubleMLIRM
* Interactive IV regression models (IIVM) - DoubleIIVM

No caso do nosso exemplo, vamos utilizar o DoubleMLPLR.

In [59]:
from doubleml import DoubleMLData, DoubleMLPLR
from sklearn.ensemble import GradientBoostingRegressor
import pandas as pd
from sklearn.model_selection import KFold

# Definir X, D, y 
X = ['casada', 'mage', 'medu', 'fhisp', 'mhisp', 'foreign', 'alcohol', 'deadkids', 'nprenatal', 'mrace', 'frace', 'fage', 'fedu']  
D = "Treated"  
y = "Y" 

# Configurando o DoubleMLData
dml_data = DoubleMLData(df, y_col=y, d_cols=D, x_cols=X)

# Ajustar Gradient Boosting com random_state para garantir reprodutibilidade (learner / Aprendiz)
ml_gb = GradientBoostingRegressor(n_estimators=100, random_state=123)

# Definir a seed na validação cruzada
kf = KFold(n_splits=5, shuffle=True, random_state=123)

# Configurar o estimador Double Machine Learning para Regressão Linear Parcial (PLR)
dml_plr = DoubleMLPLR(dml_data, ml_gb, ml_gb, n_folds=5).fit()

# Exibir o resultado do DML
print(dml_plr)

================== DoubleMLPLR Object ==================

------------------ Data summary      ------------------
Outcome variable: Y
Treatment variable(s): ['Treated']
Covariates: ['casada', 'mage', 'medu', 'fhisp', 'mhisp', 'foreign', 'alcohol', 'deadkids', 'nprenatal', 'mrace', 'frace', 'fage', 'fedu']
Instrument variable(s): None
No. Observations: 4642

------------------ Score & algorithm ------------------
Score function: partialling out

------------------ Machine learner   ------------------
Learner ml_l: GradientBoostingRegressor(random_state=123)
Learner ml_m: GradientBoostingRegressor(random_state=123)
Out-of-sample Performance:
Regression:
Learner ml_l RMSE: [[554.57781169]]
Learner ml_m RMSE: [[0.3626963]]

------------------ Resampling        ------------------
No. folds: 5
No. repeated sample splits: 1

------------------ Fit summary       ------------------
               coef    std err         t         P>|t|       2.5 %      97.5 %
Treated -204.390708  22.513418 -9.0

Lembre que no *Partially linear regression model (PLR)* a interpretação causal depende da exogeneidade condicional, o que requer controlar variáveis de confusão. O DoubleML implementa a análise de sensibilidade em relação a confundidores omitidos.

Vamos adicionar parâmetros de sensibilidade (possiveis confundidores omitidos)
* cf_y: efeito que um confundidor não observado teria sobre a variável dependente
* cf_d:representa o impacto de um confundidor omitido sobre $D$.

Utilizamos valores hipotéticos, com base no que se acredita que os confundidores omitidos podem estar influenciando em termos percentuais $D$ ou $Y$.

In [60]:
dml_plr.sensitivity_analysis(cf_y=0.04, cf_d=0.03)
print(dml_plr.sensitivity_summary)

================== Sensitivity Analysis ==================

------------------ Scenario          ------------------
Significance Level: level=0.95
Sensitivity parameters: cf_y=0.04; cf_d=0.03, rho=1.0

------------------ Bounds with CI    ------------------
    CI lower  theta lower       theta  theta upper    CI upper
0 -294.76542  -257.688472 -204.390708  -151.092945 -114.030333

------------------ Robustness Values ------------------
   H_0     RV (%)    RVa (%)
0  0.0  12.609113  10.430682


* Estimativa Original: -208.978
* Robustez
  * H0: Assume que o efeito do tratamento é zero.
  * RV(%): O valor de 12.85% indica que, se os confundidores omitidos explicassem até 12.85% da variância da variável de tratamento $D$, a estimativa de $\theta$ perderia significância estatística.
  * RVa (%): O valor de 10.67% refere-se a uma versão ajustada do RV, levando em conta a variância da variável de resposta $Y$. Se confundidores omitidos explicassem até $10.67%$ da variância de $Y$, a significância também seria perdida.

Os resultados da análise de sensibilidade sugerem que sua estimativa causal de $-208.978$ é relativamente robusta. Para que a estimativa perca significância estatística, os confundidores omitidos teriam que explicar cerca de $12.85%$ da variância de $D$, o que indica uma certa robustez do efeito causal em relação à omissão de variáveis de confusão.

## Estimação do CATE com DML

Até agora, vimos como o Double/Debiased ML (Double Machine Learning - DML) nos permite focar na estimativa do Efeito Médio do Tratamento (ATE). No entanto, ele também pode ser usado para estimar a heterogeneidade dos efeitos do tratamento ou o Efeito Médio Condicional do Tratamento (CATE). 

Logo, o CATE para modelos DoubleMLPLR consideram uma versão ligeiramente ajustada do modelo DoubleMLPLR. Em vez de considerar um efeito de tratamento constante para todas as observações, o modelo ajustado permite um efeito diferente com base em grupos.

$$ Y = D \theta_{0} (X) + g_{0}(X) + \epsilon$$

com  $E(\epsilon|D,X)=0$, e

$$ D = m_{0}(X) + \eta$$

com $E(\eta|X)=0$.

onde $\theta_{0}(X)$ é o efeito heterogêneo do tratamento.

Passos para a análise. 
* Carregar o seu conjunto de dados e definir as variáveis relevantes ($X$,$D$,$Y$).
* Criar o spline basis usando uma covariável contínua, como você fez anteriormente com mage (ou outra variável relevante).
* Estimar o CATE usando o método cate() do DoubleMLPLR.
* Gerar o intervalo de confiança para o CATE com splines.
* Plotar os resultados comparando o efeito estimado com o intervalo de confiança, similar ao gráfico gerado no exemplo.

In [67]:
import numpy as np
import pandas as pd
import doubleml as dml
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold
import patsy
import matplotlib.pyplot as plt

D = "Treated"
y = "Y"
X = ['casada', 'medu', 'fhisp', 'mhisp', 'foreign', 'alcohol',
     'deadkids', 'nprenatal', 'mrace', 'frace', 'fage', 'fedu']

data_dml_base = dml.DoubleMLData(df, y_col=y, d_cols=D, x_cols=X)

# Define the learners for the nuisance functions
ml_g = GradientBoostingRegressor(n_estimators=500, random_state=42)
ml_m = GradientBoostingRegressor(n_estimators=500, random_state=42)
ml_l = GradientBoostingRegressor(n_estimators=500, random_state=42)

# Initialize the DoubleMLPLR object with all required learners
dml_plr_obj = dml.DoubleMLPLR(data_dml_base, ml_l=ml_l, ml_m=ml_m, ml_g=ml_g)

# Fit the model
dml_plr_obj.fit()

# Print the summary
print(dml_plr_obj.summary)

A learner ml_g has been provided for score = "partialling out" but will be ignored. "A learner ml_g is not required for estimation.


               coef    std err         t         P>|t|       2.5 %      97.5 %
Treated -189.895808  23.189266 -8.188953  2.635087e-16 -235.345935 -144.445681


In [77]:
import numpy as np
import pandas as pd
import patsy
import doubleml as dml
from sklearn.ensemble import RandomForestRegressor

# Definir os modelos de machine learning
ml_g = RandomForestRegressor(n_estimators=100, max_features=20, max_depth=5, min_samples_leaf=2)
ml_m = RandomForestRegressor(n_estimators=100, max_features=20, max_depth=5, min_samples_leaf=2)

# Definir as variáveis
y_col = 'Y'
d_col = 'Treated'
x_cols = ['casada', 'mage', 'medu', 'fhisp', 'mhisp', 'foreign',
          'alcohol', 'deadkids', 'nprenatal', 'mrace', 'frace', 'fage', 'fedu']

# Preparar os dados para o DoubleML
dml_data = dml.DoubleMLData(df, y_col=y_col, d_cols=d_col, x_cols=x_cols)

# Criar o objeto DoubleMLPLR
dml_plr_obj = dml.DoubleMLPLR(dml_data, ml_g, ml_m)

# Ajustar o modelo
_ = dml_plr_obj.fit()

# Criar a matriz de design com funções base spline para 'mage'
design_matrix = patsy.dmatrix("bs(x, df=5, degree=2)", {"x": df["mage"]})
spline_basis = pd.DataFrame(design_matrix)

#print(spline_basis.head())

# Calcular o CATE usando a base spline
cate_obj = dml_plr_obj.cate(basis=spline_basis)

# Calcular os intervalos de confiança
ci = cate_obj.confint(basis=spline_basis)

print(ci.head())

        2.5 %      effect      97.5 %
0 -333.531320 -244.434154 -155.336988
1 -203.806713 -117.703031  -31.599349
2 -261.173404 -189.737681 -118.301958
3 -314.616845 -241.854760 -169.092674
4 -203.806713 -117.703031  -31.599349


A coluna effect representa a estimativa pontual do efeito do tratamento para cada observação.

In [80]:
cate_coefs = cate_obj.coef
cate_estimates = spline_basis.values @ cate_coefs

import numpy as np

cate_variances = np.sum((spline_basis.values @ cate_obj.covariance) * spline_basis.values, axis=1)
cate_se = np.sqrt(cate_variances)

from scipy.stats import norm

z_value = norm.ppf(0.975)  # Valor z para um intervalo de confiança de 95%
ci_lower = cate_estimates - z_value * cate_se
ci_upper = cate_estimates + z_value * cate_se

cate_df = pd.DataFrame({
    'mage': df['mage'],
    'cate': cate_estimates,
    'cate_lower': ci_lower,
    'cate_upper': ci_upper
})

# Ordenar os dados por 'mage' para plotagem
cate_df_sorted = cate_df.sort_values('mage')

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(cate_df_sorted['mage'], cate_df_sorted['cate'], label='CATE Estimado')
plt.fill_between(cate_df_sorted['mage'], cate_df_sorted['cate_lower'], cate_df_sorted['cate_upper'],
                 color='gray', alpha=0.2, label='IC 95%')
plt.xlabel('Idade Materna (mage)')
plt.ylabel('Efeito do Tratamento Estimado')
plt.title('Estimativas do CATE em Função da Idade Materna')
plt.legend()
plt.show()


AttributeError: 'DoubleMLBLP' object has no attribute 'coef'

Um spline é uma função matemática utilizada para aproximar ou interpolar dados, permitindo modelar relações não lineares de forma flexível. Em essência, splines são combinações de funções polinomiais de baixo grau (como polinômios de grau 2 ou 3) que são unidas em pontos específicos chamados de nós (knots). Esses nós dividem o domínio da variável independente em diferentes intervalos, e em cada intervalo, o spline pode assumir uma forma polinomial diferente, garantindo suavidade nas junções.

No contexto da sua análise com o Double Machine Learning (DML), os splines são usados para modelar a heterogeneidade do efeito do tratamento em relação a uma covariável contínua, como a idade materna (mage). Ao utilizar splines, você permite que o efeito estimado do tratamento varie de maneira flexível ao longo dos valores de mage, capturando possíveis relações não lineares entre a idade materna e o efeito do tratamento.

Você está criando uma base spline utilizando a função bs() do pacote patsy, que gera splines básicos (basis splines). Aqui, df=5 especifica o número de graus de liberdade, o que influencia o número de nós e, consequentemente, a flexibilidade do spline. degree=2 indica que está utilizando splines quadráticos. A base spline resultante é então utilizada para estimar o CATE, permitindo que o efeito do tratamento varie de forma suave e não linear com mage.

### É possivel utilizar DML com dados em painel?

Vamos considerar o efeito do homogêneo do tratamento em um painel de dados. No estilo de Sant´Anna e Zhao (2020).

Os modelos de diferença em diferenças (DID) implementados no pacote focam no caso de tratamento binário com dois períodos de tratamento. Adotando a notação de Sant'Anna e Zhao (2020) , deixe ser $Y_{it}$ o resultado de interesse para a unidade $i$ no tempo $t$. Além disso, deixe $D_{it}=1$ indicar se unidade $i$ é tratada antes do tempo $t$ (de outra forma $D_{it}=0$). Como todas as unidades começam como não tratadas ($D_{it}=0$), definir $D_{i0}=0$. Com base na notação de resultado potencial, denote $Y_{it}(1)$ como resultado da unidade $i$ no tempo $t$ se a unidade não recebeu tratamento até o momento $t$ e analogamente para $Y_{it}(0)$ com tratamento. Consequentemente, o resultado observado para a unidade $i$ no tempo $t$ é $Y_{it}=Y_{it}(1)D_{it}+Y_{it}(0)(1-D_{it})$. Além disso, deixe $X_{it}$ ser um vetor de covariáveis ​​de pré-tratamento.

O parâmetro de interesse é o efeito médio do tratamento no indivíduo tratado (ATTE). 

$$ \theta_{ATTE} = E[Y_{i1}(1) - Y_{i1}(0)|D_{it}=1]$$

As suposições de identificação correspondentes são:
* (Cond.) Tendências paralelas: $Y_{it}(1), Y_{it}(0) \perp D_{it}|X_{it}$ para $t=1,2$.
* Sobreposição: $0 < P(D_{it}=1|X_{it}) < 1$ para $t=1,2$.


Se os dados do painel estiverem disponíveis, as observações são consideradas iid. de forma ($Y_{i0},Y_{i1},D_{i},X_{i}$). Obseve que a diferença $\Delta Y_{i} = Y_{i1} - Y_{i0}$ tem que ser definida como o resultado yno DoubleMLDataobjeto.

O DoubleMLIDID implementa modelos de diferença em diferenças para dados de painel. A estimativa é conduzida por meio de seu fit() método:


In [1]:
import numpy as np
import doubleml as dml
from doubleml.datasets import make_did_SZ2020
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

ml_g = RandomForestRegressor(n_estimators=100, max_depth=5, min_samples_leaf=5)
ml_m = RandomForestClassifier(n_estimators=100, max_depth=5, min_samples_leaf=5)

np.random.seed(42)
data = make_did_SZ2020(n_obs=500, return_type='DataFrame')
obj_dml_data = dml.DoubleMLData(data, 'y', 'd')
dml_did_obj = dml.DoubleMLDID(obj_dml_data, ml_g, ml_m)

print(dml_did_obj.fit())

================== DoubleMLDID Object ==================

------------------ Data summary      ------------------
Outcome variable: y
Treatment variable(s): ['d']
Covariates: ['Z1', 'Z2', 'Z3', 'Z4']
Instrument variable(s): None
No. Observations: 500

------------------ Score & algorithm ------------------
Score function: observational

------------------ Machine learner   ------------------
Learner ml_g: RandomForestRegressor(max_depth=5, min_samples_leaf=5)
Learner ml_m: RandomForestClassifier(max_depth=5, min_samples_leaf=5)
Out-of-sample Performance:
Regression:
Learner ml_g0 RMSE: [[16.27429763]]
Learner ml_g1 RMSE: [[13.35731523]]
Classification:
Learner ml_m Log Loss: [[0.66601815]]

------------------ Resampling        ------------------
No. folds: 5
No. repeated sample splits: 1

------------------ Fit summary       ------------------
       coef   std err         t     P>|t|     2.5 %    97.5 %
d -2.840718  1.760386 -1.613691  0.106595 -6.291011  0.609575


### Double Robust Machine Learning - DRML

O Double Robust Machine Learning (DRML) é uma extensão do Double Machine Learning (DML) que combina a abordagem de DML com a abordagem de dupla robustez. A dupla robustez é uma propriedade desejável em métodos de estimação causal que garante que o estimador seja consistente se pelo menos um dos modelos de previsão estiver correto. Em outras palavras, um estimador de dupla robustez é robusto a erros de especificação em um dos modelos de previsão.



In [17]:
from econml.dml import LinearDML
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold


In [18]:
X = ['casada', 'mage', 'medu', 'fhisp', 'mhisp', 'foreign', 'alcohol', 
     'deadkids', 'nprenatal', 'mrace', 'frace', 'fage', 'fedu']  
D = "Treated"  
y = "Y" 


In [19]:
model_y = GradientBoostingRegressor(n_estimators=100, random_state=123)
model_t = GradientBoostingRegressor(n_estimators=100, random_state=123)
cv = KFold(n_splits=5, shuffle=True, random_state=123)


In [21]:
est = LinearDML(model_y=model_y, model_t=model_t, cv=cv, random_state=123)
est.fit(Y=df[y], T=df[D], X=df[X])
print(est.summary())


                      Coefficient Results                       
          point_estimate  stderr zstat  pvalue ci_lower ci_upper
----------------------------------------------------------------
casada            24.012  56.382  0.426   0.67  -86.493  134.518
mage             -11.978   5.361 -2.234  0.025  -22.485   -1.471
medu              13.463  10.551  1.276  0.202   -7.216   34.143
fhisp            129.521  134.06  0.966  0.334 -133.232  392.275
mhisp            -233.06 163.038 -1.429  0.153 -552.608   86.488
foreign          337.961 139.714  2.419  0.016   64.127  611.796
alcohol           25.692  98.715   0.26  0.795 -167.785  219.169
deadkids           38.43  51.889  0.741  0.459  -63.271  140.132
nprenatal        -11.391   6.836 -1.666  0.096   -24.79    2.008
mrace           -109.595 102.214 -1.072  0.284  -309.93    90.74
frace             33.277 100.759   0.33  0.741 -164.207   230.76
fage               0.874   2.824   0.31  0.757   -4.661    6.409
fedu               -2.21 

In [49]:
from econml.dml import LinearDML
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold

# Definir D, y e W (covariáveis)
D = "Treated"
y = "Y"
W = df[['casada', 'mage', 'medu', 'fhisp', 'mhisp', 'foreign', 'alcohol',
        'deadkids', 'nprenatal', 'mrace', 'frace', 'fage', 'fedu']]

# Configurar os modelos de machine learning
model_y = GradientBoostingRegressor(n_estimators=500, random_state=42)
model_t = GradientBoostingRegressor(n_estimators=500, random_state=42)

# Definir a validação cruzada
cv = KFold(n_splits=5, shuffle=True, random_state=24)

# Instanciar o estimador LinearDML com efeito de tratamento constante
est = LinearDML(model_y=model_y,
                model_t=model_t,
                featurizer=None,
                fit_cate_intercept=True,
                linear_first_stages=False,
                cv=cv,
                random_state=24)

# Ajustar o modelo
est.fit(Y=df[y], T=df[D], X=None, W=W)

# Resumo dos resultados
print(est.summary())


Coefficient Results:  X is None, please call intercept_inference to learn the constant!
                       CATE Intercept Results                       
               point_estimate stderr zstat  pvalue ci_lower ci_upper
--------------------------------------------------------------------
cate_intercept       -207.258 22.986 -9.017    0.0  -252.31 -162.205
--------------------------------------------------------------------

<sub>A linear parametric conditional average treatment effect (CATE) model was fitted:
$Y = \Theta(X)\cdot T + g(X, W) + \epsilon$
where for every outcome $i$ and treatment $j$ the CATE $\Theta_{ij}(X)$ has the form:
$\Theta_{ij}(X) = X' coef_{ij} + cate\_intercept_{ij}$
Coefficient Results table portrays the $coef_{ij}$ parameter vector for each outcome $i$ and treatment $j$. Intercept Results table portrays the $cate\_intercept_{ij}$ parameter.</sub>


In [34]:
from econml.dml import LinearDML
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold

# Definir X, D, y 
X = ['casada', 'mage', 'medu', 'fhisp', 'mhisp', 'foreign', 'alcohol', 
     'deadkids', 'nprenatal', 'mrace', 'frace', 'fage', 'fedu']  
D = "Treated"  
y = "Y" 

# Configurar os modelos para E[Y|X] e E[D|X]
model_y = GradientBoostingRegressor(n_estimators=100, random_state=123)
model_t = GradientBoostingRegressor(n_estimators=100, random_state=123)

# Definir a validação cruzada
cv = KFold(n_splits=5, shuffle=True, random_state=123)

# Instanciar o estimador LinearDML com efeito de tratamento constante
est = LinearDML(model_y=model_y,
                model_t=model_t,
                fit_cate_intercept=True,  # Estimar o intercepto do CATE
                linear_first_stages=False,
                cv=cv,
                random_state=123)

# Ajustar o modelo
est.fit(Y=df[y], T=df[D], X=df[X])  # Definir X=None para o efeito do tratamento constante

# Resumo dos resultados
print(est.summary())


                      Coefficient Results                       
          point_estimate  stderr zstat  pvalue ci_lower ci_upper
----------------------------------------------------------------
casada            24.012  56.382  0.426   0.67  -86.493  134.518
mage             -11.978   5.361 -2.234  0.025  -22.485   -1.471
medu              13.463  10.551  1.276  0.202   -7.216   34.143
fhisp            129.521  134.06  0.966  0.334 -133.232  392.275
mhisp            -233.06 163.038 -1.429  0.153 -552.608   86.488
foreign          337.961 139.714  2.419  0.016   64.127  611.796
alcohol           25.692  98.715   0.26  0.795 -167.785  219.169
deadkids           38.43  51.889  0.741  0.459  -63.271  140.132
nprenatal        -11.391   6.836 -1.666  0.096   -24.79    2.008
mrace           -109.595 102.214 -1.072  0.284  -309.93    90.74
frace             33.277 100.759   0.33  0.741 -164.207   230.76
fage               0.874   2.824   0.31  0.757   -4.661    6.409
fedu               -2.21 

In [ ]:
from econml.dml import LinearDML
est = LinearDML()
est.fit(y, T, X=X, W=W)
est.const_marginal_effect(X)

### É possivel utilizar DML com dados em painel?

Vamos considerar o efeito do homogêneo do tratamento em um painel de dados. No estilo de Sant´Anna e Zhao (2020).

Os modelos de diferença em diferenças (DID) implementados no pacote focam no caso de tratamento binário com dois períodos de tratamento. Adotando a notação de Sant'Anna e Zhao (2020) , deixe ser $Y_{it}$ o resultado de interesse para a unidade $i$ no tempo $t$. Além disso, deixe $D_{it}=1$ indicar se unidade $i$ é tratada antes do tempo $t$ (de outra forma $D_{it}=0$). Como todas as unidades começam como não tratadas ($D_{it}=0$), definir $D_{i0}=0$. Com base na notação de resultado potencial, denote $Y_{it}(1)$ como resultado da unidade $i$ no tempo $t$ se a unidade não recebeu tratamento até o momento $t$ e analogamente para $Y_{it}(0)$ com tratamento. Consequentemente, o resultado observado para a unidade $i$ no tempo $t$ é $Y_{it}=Y_{it}(1)D_{it}+Y_{it}(0)(1-D_{it})$. Além disso, deixe $X_{it}$ ser um vetor de covariáveis ​​de pré-tratamento.

O parâmetro de interesse é o efeito médio do tratamento no indivíduo tratado (ATTE). 

$$ \theta_{ATTE} = E[Y_{i1}(1) - Y_{i1}(0)|D_{it}=1]$$

As suposições de identificação correspondentes são:
* (Cond.) Tendências paralelas: $Y_{it}(1), Y_{it}(0) \perp D_{it}|X_{it}$ para $t=1,2$.
* Sobreposição: $0 < P(D_{it}=1|X_{it}) < 1$ para $t=1,2$.


Se os dados do painel estiverem disponíveis, as observações são consideradas iid. de forma ($Y_{i0},Y_{i1},D_{i},X_{i}$). Obseve que a diferença $\Delta Y_{i} = Y_{i1} - Y_{i0}$ tem que ser definida como o resultado yno DoubleMLDataobjeto.

O DoubleMLIDID implementa modelos de diferença em diferenças para dados de painel. A estimativa é conduzida por meio de seu fit() método:

## Considerações Finais

* O principal objetivo do DML é ajustar e remover a variável de confusão de forma que a variável de interesse (tratamento) e o desfecho (resultado) fiquem "ortogonais" ou "independentes".
* DML combina métodos de aprendizado de máquina com técnicas econométricas para estimar efeitos causais.
* A técnica geralmente envolve a aplicação de aprendizado de máquina para prever tanto o tratamento quanto o desfecho usando variáveis de confusão, e então os resíduos dessas previsões são utilizados em um segundo estágio para estimar o efeito causal.
  * Primeiro Estágio: Aplicar modelos de aprendizado de máquina para prever a variável de tratamento e o resultado.
  * Segundo Estágio: Utilizar os resíduos dessas previsões em um modelo de regressão para estimar o efeito causal.





### Doubly Robust Machine Learning (DRML)

Objetivo:

DRML visa fornecer estimativas consistentes do efeito causal mesmo que uma das duas especificações de modelo (do tratamento ou do desfecho) esteja incorreta.
Abordagem:

A metodologia DRML combina a ponderação de probabilidade inversa (IPW) com métodos de regressão.
Essa abordagem é "duplamente robusta" porque, para a consistência da estimativa, é necessário que apenas um dos modelos - ou o modelo de probabilidade de tratamento (propensity score) ou o modelo de desfecho (outcome model) - esteja corretamente especificado.
Implementação:

Modelagem do Propensity Score: Estimar a probabilidade de cada unidade receber o tratamento.
Modelagem do Desfecho: Estimar o desfecho esperado condicional ao tratamento e às covariáveis.
Combinação dos Modelos: Utilizar uma combinação das estimativas dos dois modelos para obter uma estimativa do efeito causal que é robusta a erros na especificação de um dos modelos.

#### Resumo Comparativo: DML vs DRML

* DML: Foca na remoção de viés através de uma abordagem em dois estágios, utilizando previsões de aprendizado de máquina e resíduos. Busca a ortogonalidade entre o tratamento e o desfecho.
* DRML: Utiliza a ponderação de probabilidade inversa e a modelagem do desfecho para obter estimativas robustas. É duplamente robusta porque requer que apenas um dos dois modelos esteja corretamente especificado.

Ambas as metodologias são úteis em inferência causal e se beneficiam da flexibilidade e poder dos métodos de aprendizado de máquina, mas cada uma tem suas especificidades e contextos de aplicação onde são mais adequadas.